In [1]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.8 MB/s  0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.2
    Uninstalling pip-25.2:
      Successfully uninstalled pip-25.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas geopandas geopy shapely openpyxl censusgeocode

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import geopandas as gpd
#from censusgeocode import CensusGeocode
from shapely.geometry import Point
import time

df = pd.read_excel('LWV_members.xlsx')
df.head()

,last_name,first_name,address1,address2,city,state,zip,primary_email,membership_status_description
0,Abbott,Kathy,270 E Highland Ave,Unit 1041,Milwaukee,WI,53202,Kathy.d.abbott@gmail.com,Active
1,Adams,Joy A.,3215 E Thompson Ave,NaN,Saint Francis,WI,53235-4915,joy.adams@sbcglobal.net,Active
2,Adsen,Annemarie,702 Aspen St,NaN,South Milwaukee,WI,53172,annemarieturtle@outlook.com,Active
3,Aelion,Elizabeth,210 W. Bergen Court,NaN,Fox Point,WI,53217,aelionelizabeth@gmail.com,Active
4,Albers,Lisa,6403 W Moltke Ave,NaN,Milwaukee,WI,53210,albers.lisa.m@gmail.com,Expired


In [6]:
import pandas as pd
import requests
import time

def geocode_census_api(address, zip_code):
    """Geocode using Census API directly"""
    url = "https://geocoding.geo.census.gov/geocoder/locations/onelineaddress"
    params = {
        'address': f"{address}, {zip_code}",
        'benchmark': 'Public_AR_Current',
        'format': 'json'
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        
        if data['result']['addressMatches']:
            coords = data['result']['addressMatches'][0]['coordinates']
            return coords['y'], coords['x']  # lat, lon
        return None, None
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None


# Geocode each address
results = []
for idx, row in df.iterrows():
    if idx % 10 == 0:
        print(f"Processing {idx}/{len(df)}...")
    
    lat, lon = geocode_census_api(row['address1'], row['zip'])
    results.append({'latitude': lat, 'longitude': lon})
    
    time.sleep(0.1)  # Be polite to the API

# Add coordinates to dataframe
df[['latitude', 'longitude']] = pd.DataFrame(results)

# Continue with geopandas spatial join
import geopandas as gpd
from shapely.geometry import Point

# Remove rows that couldn't be geocoded
df_valid = df.dropna(subset=['latitude', 'longitude'])

# Convert to GeoDataFrame
geometry = [Point(xy) for xy in zip(df_valid['longitude'], df_valid['latitude'])]
gdf = gpd.GeoDataFrame(df_valid, geometry=geometry, crs='EPSG:4326')# Read districts and perform spatial join
districts = gpd.read_file('Waukesha_geo/Supervisory_Districts_Adopted_11_9_21.shp')
#districts = gpd.read_file('MKE_county_wards/MilwaukeeCounty_VotingWards.shp')
gdf = gdf.to_crs(districts.crs)
result = gpd.sjoin(gdf, districts, how='left', predicate='within')

# Save results
result.to_excel('addresses_with_districts_wc.xlsx', index=False)


Processing 0/776...
Processing 10/776...
Processing 20/776...
Processing 30/776...
Processing 40/776...
Processing 50/776...
Processing 60/776...
Processing 70/776...
Processing 80/776...
Processing 90/776...
Processing 100/776...
Processing 110/776...
Processing 120/776...
Processing 130/776...
Processing 140/776...
Processing 150/776...
Processing 160/776...
Processing 170/776...
Processing 180/776...
Processing 190/776...
Processing 200/776...
Processing 210/776...
Processing 220/776...
Processing 230/776...
Processing 240/776...
Processing 250/776...
Processing 260/776...
Processing 270/776...
Processing 280/776...
Processing 290/776...
Processing 300/776...
Processing 310/776...
Processing 320/776...
Processing 330/776...
Processing 340/776...
Processing 350/776...
Processing 360/776...
Processing 370/776...
Processing 380/776...
Processing 390/776...
Processing 400/776...
Processing 410/776...
Processing 420/776...
Processing 430/776...
Processing 440/776...
Processing 450/776...

In [7]:
result.head()

,last_name,first_name,address1,address2,city,state,zip,primary_email,membership_status_description,latitude,...,geometry,index_right,OBJECTID,CNTY_FIPS,COUSUBFP,DISTRICT,Supervisor,Phone,ShapeSTAre,ShapeSTLen
0,Abbott,Kathy,270 E Highland Ave,Unit 1041,Milwaukee,WI,53202,Kathy.d.abbott@gmail.com,Active,43.044764,...,POINT (2527430.59 387792.667),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Adams,Joy A.,3215 E Thompson Ave,NaN,Saint Francis,WI,53235-4915,joy.adams@sbcglobal.net,Active,42.971945,...,POINT (2538651.696 361528.603),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adsen,Annemarie,702 Aspen St,NaN,South Milwaukee,WI,53172,annemarieturtle@outlook.com,Active,42.921474,...,POINT (2542625.787 343229.763),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aelion,Elizabeth,210 W. Bergen Court,NaN,Fox Point,WI,53217,aelionelizabeth@gmail.com,Active,43.172571,...,POINT (2526086.226 434354.822),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Albers,Lisa,6403 W Moltke Ave,NaN,Milwaukee,WI,53210,albers.lisa.m@gmail.com,Expired,43.074385,...,POINT (2504944.542 398038.916),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
nulldf = result[result['DISTRICT'].isnull()]
nulldf

,last_name,first_name,address1,address2,city,state,zip,primary_email,membership_status_description,latitude,...,geometry,index_right,OBJECTID,CNTY_FIPS,COUSUBFP,DISTRICT,Supervisor,Phone,ShapeSTAre,ShapeSTLen
0,Abbott,Kathy,270 E Highland Ave,Unit 1041,Milwaukee,WI,53202,Kathy.d.abbott@gmail.com,Active,43.044764,...,POINT (2527430.59 387792.667),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Adams,Joy A.,3215 E Thompson Ave,NaN,Saint Francis,WI,53235-4915,joy.adams@sbcglobal.net,Active,42.971945,...,POINT (2538651.696 361528.603),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adsen,Annemarie,702 Aspen St,NaN,South Milwaukee,WI,53172,annemarieturtle@outlook.com,Active,42.921474,...,POINT (2542625.787 343229.763),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aelion,Elizabeth,210 W. Bergen Court,NaN,Fox Point,WI,53217,aelionelizabeth@gmail.com,Active,43.172571,...,POINT (2526086.226 434354.822),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Albers,Lisa,6403 W Moltke Ave,NaN,Milwaukee,WI,53210,albers.lisa.m@gmail.com,Expired,43.074385,...,POINT (2504944.542 398038.916),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
769,Zellmer,Sandra,2472 N 65th St,NaN,Wauwatosa,WI,53213,zelltosa@sbcglobal.net,Expired,43.063861,...,POINT (2504653.133 394195.426),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
770,Ziegler,Connie,4822 S 19TH ST,NaN,MILWAUKEE,WI,53221,zuhleika@gmail.com,Expired,42.955131,...,POINT (2520535.399 354943.255),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
771,Ziller,Jean,2246 W Edgerton Ave,NaN,Milwaukee,WI,53221,Jeanzmke@gmail.com,Active,42.952160,...,POINT (2519152.13 353825.902),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
774,Ziolkowski,Amy,3005 E Kenwood Blvd,NaN,Milwaukee,WI,53211,amy.l.ziolkowski@gmail.com,Active,43.074570,...,POINT (2536682.09 398893.348),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
result.shape

(740, 17)

In [9]:
#mke_df = result[result['Muni'].notnull()]
#mke_df.shape
wauk_df = result[result['DISTRICT'].notnull()]
wauk_df.shape

(108, 21)

In [10]:
#mke_df.rename(columns={'index_right': 'index_right_original', 'geometry': 'geometry_addr'}, inplace=True)
#mke_df.head()
wauk_df.rename(columns={'index_right': 'index_right_original', 'geometry': 'geometry_addr'}, inplace=True)
wauk_df.head()

/var/folders/zr/7cp33krj3cgc1w_ll_m6688r0000gp/T/ipykernel_41563/312263666.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wauk_df.rename(columns={'index_right': 'index_right_original', 'geometry': 'geometry_addr'}, inplace=True)


,last_name,first_name,address1,address2,city,state,zip,primary_email,membership_status_description,latitude,...,geometry_addr,index_right_original,OBJECTID,CNTY_FIPS,COUSUBFP,DISTRICT,Supervisor,Phone,ShapeSTAre,ShapeSTLen
5,Allen,Brenda,21575 Sierra Drive,NaN,Brookfield,WI,53045,bleaallen@gmail.com,Active,43.047401,...,POINT (2454796.615 387050.576),9.0,10.0,55133,COUNTY,10,Terry Thieme\r\n,None,4.069878e+08,124516.462132
6,Allen,Rhea,21575 Sierra Drive,NaN,Brookfield,WI,53045,rhealea0830@gmail.com,Active,43.047401,...,POINT (2454796.615 387050.576),9.0,10.0,55133,COUNTY,10,Terry Thieme\r\n,None,4.069878e+08,124516.462132
7,Allison,Shirley,2243A Circle Ridge,NaN,Delafield,WI,53018,shirleyallison@wi.rr.com,Active,43.054276,...,POINT (2401624.403 388459.867),11.0,12.0,55133,COUNTY,12,Peter M. Wolff\r\n,None,1.046186e+09,181889.911806
12,Anderson,Arthur,1285 N 124th St,NaN,Elm Grove,WI,53122,ajohnanderson@gmail.com,Expired,43.046895,...,POINT (2485164.241 387549.777),7.0,8.0,55133,COUNTY,8,Johnny Koremenos,None,2.481750e+08,82006.644681
16,Anderson,Jayden,3725 Mountain Dr,NaN,Brookfield,WI,53045-1442,nedyajx2@gmail.com,Active,43.086221,...,POINT (2465246.697 401433.304),8.0,9.0,55133,COUNTY,9,Jim Heinrich\r\n,None,3.660013e+08,113723.724491


In [11]:
#mke_df['Muni'] = mke_df['Muni'].astype(int)
wauk_df['DISTRICT'] = wauk_df['DISTRICT'].astype(int)

/Users/mafphd/lwv/.venv/lib/python3.12/site-packages/geopandas/geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [12]:
wauk_df.to_excel('waukesha_addresses_with_districts.xlsx', index=False)

In [20]:
rep_map_df = gpd.read_file('MKE_rep_geo/City_Common_Council_District_or_Village_Board_of_Trustees_in_Milwaukee_County.shp', )
rep_map_df.head()

,District_D,Member_Nam,Email_Addr,Phone_Nbr,Member_Url,Muni_Code,Council_Di,geometry
0,City of Cudahy Common Council District 1,Miranda Levy,mailto:mlevy@ci.cudahy.wi.us?subject=,414-426-5471,https://www.cudahy-wi.gov/government/common_co...,17975,1,"POLYGON ((2540177.062 356888.001, 2538572.804 ..."
1,City of Cudahy Common Council District 2,Jason Phillips,mailto:jphillips@ci.cudahy.wi.us?subject=,404-414-4729,https://www.cudahy-wi.gov/government/common_co...,17975,2,"POLYGON ((2543777.631 350528.909, 2543774.838 ..."
2,City of Cudahy Common Council District 3,Rob Haines,mailto:rhaines@ci.cudahy.wi.us?subject=,414-524-9962,https://www.cudahy-wi.gov/government/common_co...,17975,3,"POLYGON ((2541459.45 351004.851, 2541459.231 3..."
3,City of Cudahy Common Council District 4,"Kenneth R. Jankowski, Jr.",mailto:kjankowski@ci.cudahy.wi.us?subject=,414-469-6099,https://www.cudahy-wi.gov/government/common_co...,17975,4,"POLYGON ((2534113.527 346134.489, 2534116.857 ..."
4,City of Cudahy Common Council District 5,Michele St. Marie-Boelkow,mailto:st.marie-boelkowm@ci.cudahy.wi.us?subject=,414-483-2489,https://www.cudahy-wi.gov/government/common_co...,17975,5,"POLYGON ((2541344.586 346344.139, 2541329.542 ..."


In [69]:
rep_map_df.rename(columns = {'Member_Nam': 'incumbent'}, inplace=True)
rep_map_df.columns

Index(['District_D', 'incumbent', 'Email_Addr', 'Phone_Nbr', 'Member_Url',
       'Muni_Code', 'Council_Di', 'geometry'],
      dtype='object')

In [42]:
print(rep_map_df.crs)
print(mke_df.crs)

EPSG:6609
EPSG:6609


In [70]:

# --- Inputs ---
districts_gdf = rep_map_df.copy()   # municipal polygons, already EPSG:6609 per your print
addresses_any = mke_df.set_geometry('geometry_addr')           # either GeoDataFrame with geometry, or DataFrame with lat/lon

# --- Confirm CRS ---
print("Districts CRS:", districts_gdf.crs)  # should be EPSG:6609
print("Addresses type:", type(addresses_any))
print("Addresses CRS:", getattr(addresses_any, "crs", None))

# --- Build addresses GeoDataFrame correctly ---
if isinstance(addresses_any, gpd.GeoDataFrame) and addresses_any.geometry is not None:
    # Case A: already a GeoDataFrame
    addresses_gdf = addresses_any.copy()
    if addresses_gdf.crs is None:
        raise ValueError("addresses_gdf has geometry but no CRS; set the correct EPSG (6609 or 4326).")
    # If addresses are not in 6609, reproject to 6609
    if addresses_gdf.crs.to_epsg() != 6609:
        addresses_gdf = addresses_gdf.to_crs("EPSG:6609")
else:
    # Case B: DataFrame with lat/lon (in degrees)
    # 1) create points in EPSG:4326
    addresses_gdf = gpd.GeoDataFrame(
        addresses_any.copy(),
        geometry=gpd.points_from_xy(addresses_any["longitude"], addresses_any["latitude"]),
        crs="EPSG:4326"
    )
    # 2) reproject points to EPSG:6609 to match districts
    addresses_gdf = addresses_gdf.to_crs("EPSG:6609")

# --- Ensure districts_gdf CRS is set (it is, but guard anyway) ---
if districts_gdf.crs is None:
    raise ValueError("districts_gdf.crs is None—set it to 'EPSG:6609' (or the known CRS).")

# --- Sanity check: bounds should overlap ---
print("Addresses bounds (6609):", addresses_gdf.total_bounds)
print("Districts bounds (6609):", districts_gdf.total_bounds)

# --- Spatial join: point-in-polygon ---
joined = gpd.sjoin(
    addresses_gdf,
    districts_gdf,
    how="left",
    predicate="within",
    lsuffix="_addr",
    rsuffix="_poly"
)

# --- Verify the district column exists on the right frame ---
print("District columns in polygons:", [c for c in districts_gdf.columns if c != "geometry"])
assert "District_D" in districts_gdf.columns, "Column 'District_D' not found in districts_gdf."

# --- Columns to keep (only keep those that exist to avoid KeyError) ---
desired_cols = [
    'last_name', 'first_name', 'address1', 'address2', 'city', 'state',
    'zip', 'primary_email', 'membership_status_description',
    'Ward_ID', 'AlderID', 'SuperID', 'Muni', 'latitude', 'longitude',
    'District_D', 'incumbent', 'geometry'  # geometry of the address point in EPSG:6609
]
cols_to_keep = [c for c in desired_cols if c in joined.columns]
output = joined[cols_to_keep]

output.head()


Districts CRS: EPSG:6609
Addresses type: <class 'geopandas.geodataframe.GeoDataFrame'>
Addresses CRS: EPSG:6609
Addresses bounds (6609): [2485417.67746102  322953.68959607 2543917.80405325  440988.68340773]
Districts bounds (6609): [2483454.26999588  313358.15735573 2552076.40801381  441558.22969523]
District columns in polygons: ['District_D', 'incumbent', 'Email_Addr', 'Phone_Nbr', 'Member_Url', 'Muni_Code', 'Council_Di']


,last_name,first_name,address1,address2,city,state,zip,primary_email,membership_status_description,Ward_ID,AlderID,SuperID,Muni,latitude,longitude,District_D,incumbent
0,Abbott,Kathy,270 E Highland Ave,Unit 1041,Milwaukee,WI,53202,Kathy.d.abbott@gmail.com,Active,158,4,3.0,53000,43.044764,-87.908876,City of Milwaukee Common Council District 4,Robert Bauman
1,Adams,Joy A.,3215 E Thompson Ave,NaN,Saint Francis,WI,53235-4915,joy.adams@sbcglobal.net,Active,8,2,8.0,70650,42.971945,-87.869423,City of St. Francis Common Council District 2,Donald Brickner
1,Adams,Joy A.,3215 E Thompson Ave,NaN,Saint Francis,WI,53235-4915,joy.adams@sbcglobal.net,Active,8,2,8.0,70650,42.971945,-87.869423,City of St. Francis Common Council District 2,Shawn Feirer
2,Adsen,Annemarie,702 Aspen St,NaN,South Milwaukee,WI,53172,annemarieturtle@outlook.com,Active,2,1,8.0,75125,42.921474,-87.856336,City of South Milwaukee Common Council District 1,Brett Briesemeister
2,Adsen,Annemarie,702 Aspen St,NaN,South Milwaukee,WI,53172,annemarieturtle@outlook.com,Active,2,1,8.0,75125,42.921474,-87.856336,City of South Milwaukee Common Council District 1,Craig Maass


In [71]:
output['District_D'].isnull().sum()

np.int64(0)

In [72]:
print(output.shape)
output = output.drop_duplicates(subset=["latitude", "longitude"], keep="first")
print(output.shape)

(1374, 17)
(488, 17)


In [79]:

# 1) First word (e.g., "City", "Town", "Village")
# - Strip leading/trailing whitespace, then extract the first alphabetic token.
output["Muni_type"] = (
    output["District_D"]
        .astype(str)
        .str.strip()
        .str.extract(r'^([A-Za-z-]+)', expand=False)  # allow hyphenated like "Town-Village" if needed
)


last_parts = (
    output["District_D"]
        .astype(str)
        .str.strip()
        .str.extract(r'([A-Za-z-]+)(?:\s+(\d+))?$', expand=True)  # small tweak to require space before digits
)

# Build Rep_type: "Word N" if N exists, else "Word"
output["Rep_type"] = last_parts[0].str.cat(last_parts[1], sep=" ", na_rep="")


# Quick preview
output[["District_D", "Muni_type", "Rep_type"]].head()


,District_D,Muni_type,Rep_type
0,City of Milwaukee Common Council District 4,City,District 4
1,City of St. Francis Common Council District 2,City,District 2
2,City of South Milwaukee Common Council District 1,City,District 1
3,Village of Fox Point Board of Trustees,Village,Trustees
4,City of Milwaukee Common Council District 10,City,District 10


In [80]:
output.columns

Index(['last_name', 'first_name', 'address1', 'address2', 'city', 'state',
       'zip', 'primary_email', 'membership_status_description', 'Ward_ID',
       'AlderID', 'SuperID', 'Muni', 'latitude', 'longitude', 'District_D',
       'incumbent', 'Muni_type', 'last_word', 'Rep_type'],
      dtype='object')

In [82]:
output.drop(columns = 'last_word').to_excel('mke_county_list.xlsx', index=False)